# IFC OpenShell Voice Assistant - Prototype #5


In [ ]:
%%html
<iframe src="https://ifcjs.github.io/web-ifc-viewer/example/index" width="500" height="250"></iframe>

In [ ]:
import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re
import pyttsx3
import webbrowser

sensitive_keywords = ["colour", "dimension", "element", "file"]
colours = ["red","blue","yellow"]
dimensions = ["x","y","height"]
elements = ["wall","window","door"]
files = ["4","5","6"]


def text_to_voice (text):

    engine = pyttsx3.init();
    engine.say(text);
    # engine.setProperty('voice', voices[2].id)
    engine.setProperty("rate", 155)
    engine.runAndWait() ;


def get_voice():

    raw_text = {}
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)
        r.pause_threshold = 0.8
        r.dynamic_energy_threshold = True
        audio = r.listen(source)
        raw_text = r.recognize_google(audio)
    return raw_text


def nlp(raw_text, sensitive_keywords):

    command_type = [x for x in sensitive_keywords if x in raw_text]
    if command_type[0] == 'colour':
        command_sup = [x for x in colours if x in raw_text]
    elif command_type[0] == 'dimension':
        command_sup = [x for x in dimensions if x in raw_text]
    elif command_type[0] == 'element':
        command_sup = [x for x in elements if x in raw_text]
    elif command_type[0] == 'file':
        command_sup = [x for x in files if x in raw_text]
    return command_type, command_sup


def open_file():

    text_to_voice (f' Can you please give me the file name')
    get_voice_text = get_voice()
    file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]

    if [x for x in file_name_ if x in files]:
        
        text_to_voice (f'Ok, I will open the file {file_name_}')
        webbrowser.open(f'{file_name_}.txt')

    else:
        
        while bool([x for x in file_name_ if x in files])==False:
            
            text_to_voice (f' Sorry, but I can only open file {files}, Can you please give me the file name?')
            get_voice_text = get_voice()
            file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]
            if [x for x in file_name_ if x in files]:
                text_to_voice (f'Ok, I will open the file {file_name_}')
                webbrowser.open(f'{file_name_}.txt')

def change_wall_color():
    
    text_to_voice (f' Can you please give me the desired colour')
    get_voice_text = get_voice()
    colour_name_ = [x for x in get_voice_text if x in colours]

    if [x for x in colours if x in get_voice_text]:
        
        text_to_voice (f'Ok, I will change the colour to {colour_name_}')
        """a command to change the colour of the wall"""
        
    else:
        
        while bool([x for x in colours if x in get_voice_text])==False:
            
            text_to_voice (f' Sorry, but I can only change to colour to {colours},Can you please give me the desired colour?')
            get_voice_text = get_voice()
            colour_name_ = [x for x in colours if x in get_voice_text]
            
            if [x for x in colour_name_ if x in colours]:
                
                text_to_voice (f'Ok, I will change the colour to {colour_name_}')
                """a command to change the colour of the wall"""


             
def change_dimension():
    pass


def create_element():
    pass


def get_action(command_type, command_sup):

    if command_type[0] == 'colour':
        change_wall_color()
        
    elif command_type[0] == 'dimension':
        change_dimension()
        
    elif command_type[0] == 'element':
        create_element()
        
    elif command_type[0] == 'file':
        open_file()           

In [ ]:
text_to_voice ('Hi')

In [ ]:
raw_text = get_voice()

In [ ]:
raw_text

In [ ]:
colours

In [ ]:
colour_name = [x for x in colours if x in raw_text]
colour_name

In [ ]:
command_type, command_sup = nlp(raw_text, sensitive_keywords)

In [ ]:
get_action(command_type, command_sup)